## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64
import pytz
import pandas as pd

from IPython.display import HTML

# import sys
# sys.path.append('../../')

from openbb_terminal.reports import widget_helpers as widgets
from openbb_terminal.sdk import openbb
from openbb_terminal.sdk import helper
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal.core.config.paths import (
    REPOSITORY_DIRECTORY,
    USER_PORTFOLIO_DATA_DIRECTORY,
)
from openbb_terminal import OpenBBFigure

warnings.filterwarnings("ignore")

# Detect if prediction capabilities are present. If they are not, disable prediction in the rest of the script
# so that the report can still be generated without prediction results.
# predictions = True
# try:
#     openbb.stocks.pred.models
# except Exception as e:
#     predictions = False

# TODO Fix predictions virtual path on api refactored

predictions = False

### Header

In [ ]:
theme = TerminalStyle("light", "light", "light")
stylesheet = widgets.html_report_stylesheet()
user_time_zone = pytz.timezone(helper.get_user_timezone())

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
transactions = "holdings_example.xlsx"
report_name = "Portfolio Report"

In [ ]:
file_0 = (
    str(REPOSITORY_DIRECTORY)
    + "/openbb_terminal/miscellaneous/portfolio/"
    + transactions
)
file_1 = str(USER_PORTFOLIO_DATA_DIRECTORY) + "/holdings/" + transactions

if os.path.isfile(file_0):
    transactions_path = file_0
elif os.path.isfile(file_1):
    transactions_path = file_1
else:
    raise FileNotFoundError("File not found: " + transactions)

In [ ]:
_, transactions_file = os.path.split(transactions_path)
report_title = f"Portfolio report for {transactions_file}"
report_date = pd.Timestamp.now(tz=user_time_zone).strftime("%d %B, %Y")
report_time = pd.Timestamp.now(tz=user_time_zone).strftime("%H:%M")
report_timezone = pd.Timestamp.now(tz=user_time_zone).tzinfo
report_title, report_date, report_time, report_timezone

## Data

Loading

In [ ]:
try:
    P = openbb.portfolio.load(transactions_path)
    openbb.portfolio.show(P)
except Exception as e:
    raise Exception("Failed to load the transactions.")

In [ ]:
beta_chart = openbb.portfolio.rbeta_chart(P, external_axes=True)

if beta_chart:
    beta_chart = beta_chart.to_html()

In [ ]:
sharpe_chart = openbb.portfolio.rsharpe_chart(P, external_axes=True)

if sharpe_chart:
    sharpe_chart = sharpe_chart.to_html()

In [ ]:
volatility_chart = openbb.portfolio.rvol_chart(P, external_axes=True)

if volatility_chart:
    volatility_chart = volatility_chart.to_html()

In [ ]:
sortino_chart = openbb.portfolio.rsort_chart(P, external_axes=True)

if sortino_chart:
    sortino_chart = sortino_chart.to_html()

In [ ]:
maxdd_chart = openbb.portfolio.maxdd_chart(P, external_axes=True)

if maxdd_chart:
    maxdd_chart = maxdd_chart.to_html()

In [ ]:
country_allocation = openbb.portfolio.alloc.countries(P)

country_allocation["Portfolio"] = (country_allocation["Portfolio"] * 100).map(
    "{:.3}%".format
)
country_allocation["Benchmark"] = (country_allocation["Benchmark"] * 100).map(
    "{:.3}%".format
)
country_allocation["Difference"] = (country_allocation["Difference"] * 100).map(
    "{:.3}pp".format
)

country_allocation

In [ ]:
sector_allocation = openbb.portfolio.alloc.sectors(P)

sector_allocation["Portfolio"] = (sector_allocation["Portfolio"] * 100).map(
    "{:.3}%".format
)
sector_allocation["Benchmark"] = (sector_allocation["Benchmark"] * 100).map(
    "{:.3}%".format
)
sector_allocation["Difference"] = (sector_allocation["Difference"] * 100).map(
    "{:.3}pp".format
)

sector_allocation

In [ ]:
distr_chart = openbb.portfolio.distr_chart(P, external_axes=True)

if distr_chart:
    distr_chart = distr_chart.to_html()

In [ ]:
dret_chart = openbb.portfolio.dret_chart(P, external_axes=True)

if dret_chart:
    dret_chart = dret_chart.to_html()

In [ ]:
yret_chart = openbb.portfolio.yret_chart(P, external_axes=True)

if yret_chart:
    yret_chart = yret_chart.to_html()

## Render the report template to a file

In [ ]:
body = f"""
<style>
@font-face {{
    font-family: "Fira Code";
    src: url("{(OpenBBFigure.plotlyjs_path.parent.parent / "web"/"css").as_uri()}/fonts/FiraCode-Regular.ttf") format("truetype");
    font-weight: 400 700;
    font-stretch: 50%;
}}
</style>
<script src="/{OpenBBFigure.plotlyjs_path.as_uri()}"></script>
"""


img = (
    str(REPOSITORY_DIRECTORY)
    + "/openbb_terminal/reports/templates/OpenBB_reports_logo.png"
)
floppy_disk_img = (
    str(REPOSITORY_DIRECTORY) + "/openbb_terminal/reports/templates/floppy-disc.png"
)
body += widgets.header(
    img,
    floppy_disk_img,
    "OpenBB",
    report_date,
    report_time,
    report_timezone,
    report_title,
)

body += widgets.tablinks(["SUMMARY", "In-Depth Analysis", "Statistics"])

# Summary
htmlcode = widgets.p(
    "The total performance of the portfolio versus the benchmark is as follows. "
    "Please note this calculation assumes partially shares exist for the benchmark."
)

htmlcode += widgets.row(
    [widgets.h(3, "Performance") + openbb.portfolio.perf(P).to_html()]
)

htmlcode += widgets.p("In which the following yearly returns are achieved:")

htmlcode += widgets.row([widgets.h(3, "Yearly Performance") + yret_chart])

htmlcode += widgets.p(
    "The portfolio and benchmark consists of the following allocations to countries and sectors."
)

htmlcode += widgets.row(
    [widgets.h(3, "Country Allocation") + country_allocation.to_html()]
)

htmlcode += widgets.row(
    [widgets.h(3, "Sector Allocation") + sector_allocation.to_html()]
)

htmlcode += widgets.p(
    "The portfolio consists of the following trades including returns."
)

htmlcode += widgets.row(
    [widgets.h(3, "Portfolio Trades") + P.portfolio_trades.to_html()]
)

htmlcode += widgets.p(
    "The benchmark consists of the following trades including returns."
)

htmlcode += widgets.row(
    [widgets.h(3, "Benchmark Trades") + P.benchmark_trades.to_html()]
)

body += widgets.add_tab("SUMMARY", htmlcode)

# In Depth Analysis
htmlcode = widgets.p(
    "The following metrics, Tracking Error and Information Ratio, measure whether you are "
    "closely aligned with the benchmark. If the Tracking Error is not close to 1, a different benchmark "
    "might be a better fit."
)

htmlcode += widgets.row(
    [widgets.h(3, "Tracking Error") + openbb.portfolio.metric.trackerr(P)[0].to_html()]
)
htmlcode += widgets.row(
    [
        widgets.h(3, "Information Ratio")
        + openbb.portfolio.metric.information(P).to_html()
    ]
)

htmlcode += widgets.row([widgets.h(3, "Beta Chart") + beta_chart])

htmlcode += widgets.row(
    [widgets.h(3, "Sharpe Ratio") + openbb.portfolio.metric.sharpe(P).to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Sharpe Ratio Chart") + sharpe_chart])
htmlcode += widgets.row(
    [widgets.h(3, "Volatility") + openbb.portfolio.metric.volatility(P).to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Volatility Chart") + volatility_chart])
htmlcode += widgets.row(
    [widgets.h(3, "Sortino Ratio") + openbb.portfolio.metric.sortino(P).to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Sortino Chart") + sortino_chart])

htmlcode += widgets.row([widgets.h(3, "Maximum Drawdown") + maxdd_chart])


body += widgets.add_tab("In-Depth Analysis", htmlcode)

# Statistics
htmlcode = widgets.row([widgets.h(3, "Distribution") + distr_chart])
htmlcode += widgets.row([widgets.h(3, "Daily Returns") + dret_chart])
htmlcode += widgets.row(
    [widgets.h(3, "Volatility") + openbb.portfolio.metric.volatility(P).to_html()]
)
htmlcode += widgets.row(
    [widgets.h(3, "Kurtosis") + openbb.portfolio.metric.kurtosis(P).to_html()]
)
htmlcode += widgets.row(
    [widgets.h(3, "Skew") + openbb.portfolio.metric.skew(P).to_html()]
)
htmlcode += widgets.row(
    [widgets.h(3, "Value at Risk (VaR)") + openbb.portfolio.var(P).to_html()]
)
htmlcode += widgets.row(
    [widgets.h(3, "Expected Shortfall (cVaR)") + openbb.portfolio.es(P).to_html()]
)
htmlcode += widgets.row(
    [widgets.h(3, "Omega Ratio") + openbb.portfolio.om(P).to_html()]
)

body += widgets.add_tab("Statistics", htmlcode)

body += widgets.tab_clickable_and_save_evt()

report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)